In [ ]:
import fasttext
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
def process_and_split_train_csv(input_file, output_yoruba, output_english, rows=250):
    """
    Processes the train.csv file to create separate Yoruba and English CSVs.
    :param input_file: Path to the input train.csv file.
    :param output_yoruba: Path to save the Yoruba sentences CSV.
    :param output_english: Path to save the English sentences CSV.
    :param rows: Number of rows to process (default is 250).
    """
    
    train_data = pd.read_csv(input_file)
    
    processed_data = train_data[['Yoruba', 'English']].head(rows)
    
    processed_data[['Yoruba']].to_csv(output_yoruba, index=False, header=False)
    print(f"Yoruba sentences saved to {output_yoruba}")
    
    processed_data[['English']].to_csv(output_english, index=False, header=False)
    print(f"English sentences saved to {output_english}")

input_file = 'train.csv' 
output_yoruba = 'yoruba_sentences.csv'  
output_english = 'english_sentences.csv' 

process_and_split_train_csv(input_file, output_yoruba, output_english, rows=50)


In [ ]:
def load_fasttext_models():
    print("Downloading and loading FastText models...")
    en_model_path = hf_hub_download(repo_id="facebook/fasttext-en-vectors", filename="model.bin")
    en_model = fasttext.load_model(en_model_path)

    yo_model_path = hf_hub_download(repo_id="facebook/fasttext-yo-vectors", filename="model.bin")
    yo_model = fasttext.load_model(yo_model_path)

    return en_model, yo_model

def load_mbert_model():
    print("Loading mBERT model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    model = AutoModel.from_pretrained("bert-base-multilingual-cased")
    return tokenizer, model

def get_mbert_embedding(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()

def evaluate_fasttext_sentences(yo_model, en_model, yoruba_sentences, english_sentences):
    results = []
    for yoruba_sentence in yoruba_sentences:
        try:
            yo_vec = np.mean([yo_model.get_word_vector(word) for word in yoruba_sentence.split()], axis=0)
            closest_sentence = sorted(
                english_sentences,
                key=lambda en_sentence: cosine(
                    yo_vec,
                    np.mean([en_model.get_word_vector(word) for word in en_sentence.split()], axis=0)
                )
            )[0]
            similarity = 1 - cosine(
                yo_vec,
                np.mean([en_model.get_word_vector(word) for word in closest_sentence.split()], axis=0)
            )
            results.append((yoruba_sentence, closest_sentence, similarity))
        except Exception as e:
            print(f"Error processing sentence '{yoruba_sentence}': {e}")
            results.append((yoruba_sentence, "Error", 0))
    return results

def evaluate_mbert_sentences(yoruba_sentences, english_sentences, tokenizer, model):
    results = []
    for yoruba_sentence in yoruba_sentences:
        try:
            yo_vec = get_mbert_embedding(model, tokenizer, yoruba_sentence)
            closest_sentence = sorted(
                english_sentences,
                key=lambda en_sentence: cosine(
                    yo_vec,
                    get_mbert_embedding(model, tokenizer, en_sentence)
                )
            )[0]
            similarity = 1 - cosine(yo_vec, get_mbert_embedding(model, tokenizer, closest_sentence))
            results.append((yoruba_sentence, closest_sentence, similarity))
        except Exception as e:
            print(f"Error processing sentence '{yoruba_sentence}': {e}")
            results.append((yoruba_sentence, "Error", 0))
    return results

if __name__ == "__main__":
    yoruba_sentences = pd.read_csv('yoruba_sentences.csv', header=None).squeeze().tolist()
    english_sentences = pd.read_csv('english_sentences.csv', header=None).squeeze().tolist()

    en_model, yo_model = load_fasttext_models()
    tokenizer, mbert_model = load_mbert_model()

    print("Evaluating FastText with Yoruba and English sentences...")
    fasttext_sentence_results = evaluate_fasttext_sentences(yo_model, en_model, yoruba_sentences, english_sentences)

    print("Evaluating mBERT with Yoruba and English sentences...")
    mbert_sentence_results = evaluate_mbert_sentences(yoruba_sentences, english_sentences, tokenizer, mbert_model)

    sentence_comparison_df = pd.DataFrame({
        "Yoruba Sentence": yoruba_sentences,
        "FastText Match": [row[1] for row in fasttext_sentence_results],
        "FastText Similarity": [row[2] for row in fasttext_sentence_results],
        "mBERT Match": [row[1] for row in mbert_sentence_results],
        "mBERT Similarity": [row[2] for row in mbert_sentence_results]
    })
    sentence_comparison_df.to_csv("yoruba_english_sentence_comparison.csv", index=False)

    print("Sentence-level comparison complete. Results saved to 'yoruba_english_sentence_comparison.csv'.")


In [ ]:
def plot_sentence_similarity_bar_graphs(comparison_df):
    """
    Plots bar graphs comparing FastText and mBERT cosine similarity scores with generic labels.
    """
    num_sentences = len(comparison_df)
    labels = [f"Sentence {i+1}" for i in range(num_sentences)]
    fasttext_scores = comparison_df["FastText Similarity"]
    mbert_scores = comparison_df["mBERT Similarity"]

    x = np.arange(len(labels))
    width = 0.35

    plt.figure(figsize=(12, 8))
    plt.bar(x - width/2, fasttext_scores, width, label="FastText")
    plt.bar(x + width/2, mbert_scores, width, label="mBERT")

    plt.xlabel("Sentences")
    plt.ylabel("Cosine Similarity")
    plt.title("Comparison of FastText and mBERT Similarities for Sentences")
    plt.xticks(x, labels, rotation=45, ha="right")
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_sentence_similarity_bar_graphs(sentence_comparison_df)

In [ ]:
from scipy.stats import ttest_ind

comparison_df = pd.read_csv("yoruba_english_sentence_comparison.csv")

fasttext_scores = comparison_df["FastText Similarity"]
mbert_scores = comparison_df["mBERT Similarity"]

t_stat, p_value = ttest_ind(fasttext_scores, mbert_scores)
print(f"T-Statistic: {t_stat}, P-Value: {p_value}")

if p_value < 0.05:
    print("The difference between FastText and mBERT is statistically significant.")
else:
    print("The difference between FastText and mBERT is not statistically significant.")


In [ ]:
def evaluate_on_random_subset(yoruba_sentences, english_sentences, subset_size=10, num_runs=5):
    results = []
    for _ in range(num_runs):
        sampled_indices = np.random.choice(len(yoruba_sentences), subset_size, replace=False)
        sampled_yoruba = [yoruba_sentences[i] for i in sampled_indices]
        sampled_english = [english_sentences[i] for i in sampled_indices]
        
        fasttext_results = evaluate_fasttext_sentences(yo_model, en_model, sampled_yoruba, sampled_english)
        fasttext_avg_similarity = np.mean([row[2] for row in fasttext_results])
        
        mbert_results = evaluate_mbert_sentences(sampled_yoruba, sampled_english, tokenizer, mbert_model)
        mbert_avg_similarity = np.mean([row[2] for row in mbert_results])
        
        results.append((fasttext_avg_similarity, mbert_avg_similarity))
    
    return pd.DataFrame(results, columns=["FastText Avg Similarity", "mBERT Avg Similarity"])

robustness_results = evaluate_on_random_subset(yoruba_sentences, english_sentences, subset_size=10, num_runs=5)
print(robustness_results)


In [ ]:
from sklearn.decomposition import PCA

yoruba_embeddings = [get_mbert_embedding(mbert_model, tokenizer, text) for text in yoruba_sentences]
english_embeddings = [get_mbert_embedding(mbert_model, tokenizer, text) for text in english_sentences]

all_embeddings = np.vstack(yoruba_embeddings + english_embeddings)
labels = ["Yoruba"] * len(yoruba_embeddings) + ["English"] * len(english_embeddings)

pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(all_embeddings)

plt.figure(figsize=(8, 6))
for label in set(labels):
    idx = [i for i, l in enumerate(labels) if l == label]
    plt.scatter(reduced_embeddings[idx, 0], reduced_embeddings[idx, 1], label=label)
plt.legend()
plt.title("PCA Visualization of mBERT Embeddings")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()
